In [1]:
import requests
from bs4 import BeautifulSoup
from time import sleep
import pandas as pd
from transformers import pipeline
import torch

c:\Users\Tijn\Documents\VSC\DAta engineering projects\Job_scraping\.env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
job_ids = []
trigger_words = ['data', 'machine learning', 'ai', 'ml ', 'statist', 'artificial intelligence', 'python']
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
    'Accept': '*/*',
    'Accept-Language': 'en-US,en;q=0.9',
    'Accept-Encoding': 'gzip, deflate, br',
    'Connection': 'keep-alive',
    'Referer': 'https://www.linkedin.com/jobs/search/?keywords=data%2Bscientist&location=Nederland&geoId=102890719&trk=homepage-basic_jobs-search-bar_search-submit&position=1&pageNum=0',
}
for start in range(0, 250, 25):
    URL = f"https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=data%2Bscientist&location=Nederland&geoId=102890719&start={start}"
    response = requests.get(URL, headers=headers)
    if response.status_code != 200:
        print(f"Error: Unable to fetch data from LinkedIn. Status code: {response.status_code}")
        # leave the loop if the request fails
        break
    else:
        print("Data fetched successfully!")
        data = response.text
    soup = BeautifulSoup(data, 'html.parser')
    job_listings = soup.find_all('a', class_='base-card__full-link absolute top-0 right-0 bottom-0 left-0 p-0 z-[2]')


    for job in job_listings:
        if any(word in job.text.strip().lower() for word in ['intern', 'afstudeeropdracht', 'stage']):
            print(f"--- Job ID {job.text.strip()} is an intern position.")
        elif any(word in job.text.strip().lower() for word in trigger_words):
            job_ids.append(('').join(job.get('href').split('/')[5:]).split('-')[-1].split('?')[0])
            print(f"!!! Job ID {job.text.strip()} contains trigger words.")
        else:
            print(f"Job ID {job.text.strip()} does not contain trigger words.")
    print('')

print(f"Total job IDs found: {len(job_ids)}")

Data fetched successfully!
Job ID R&D Development Specialist does not contain trigger words.
Job ID Associate Scientist does not contain trigger words.
!!! Job ID Machine Learning Engineer contains trigger words.
Job ID Help je mee ??? does not contain trigger words.
--- Job ID R&D Modelling-Statistics Intern is an intern position.
--- Job ID Internship - Data Science AI/ML is an intern position.
!!! Job ID Data Scientist II - Smart Benefits Insights contains trigger words.
!!! Job ID Data Scientist contains trigger words.
!!! Job ID Data Scientist contains trigger words.
Job ID Boa, Utrecht does not contain trigger words.

Data fetched successfully!
!!! Job ID Data Scientist contains trigger words.
!!! Job ID Data Scientist AI/ML in Product Development contains trigger words.
Job ID Stanowisko ds. wsparcia procesów informatycznych w Wydziale Wsparcia Informatycznego w obszarze Warstwy Pośredniczącej KSI ZUS (TUX) CI_01/2025 does not contain trigger words.
!!! Job ID AI/Machine Learnin

In [3]:
data = {'job_title' : [], 'job_company': [], 'job_location': [], 'days_ago': [], 'company_description': [], 'job_description': []}

for job_id in job_ids:
    URL = f"https://www.linkedin.com/jobs-guest/jobs/api/jobPosting/{job_id}"
    response = requests.get(URL, headers=headers)
    if response.status_code != 200:
        print(f"Error: Unable to fetch data from LinkedIn. Status code: {response.status_code}")
    else:
        print("Data fetched successfully!")
        data2 = response.text
    soup2 = BeautifulSoup(data2, 'html.parser')
    soup2.find('div', class_= "show-more-less-html__markup show-more-less-html__markup--clamp-after-5 relative overflow-hidden")
    text = soup2.get_text(strip=True, separator="~~")
    data['job_title'].append(text.split('~~')[0])
    data['job_company'].append(text.split('~~')[1])
    data['job_location'].append(text.split('~~')[2])
    data['days_ago'].append(text.split('~~')[3].split(' ')[0])
    data['company_description'].append((' ').join(text.split('~~')[10:12]))
    job_desc = (' ').join(text.split('~~')[13:-12])
    #if job_desc[0:12] == 'Remove photo':
    #    job_desc = job_desc.split(' ')[49:]
    data['job_description'].append(job_desc)
    sleep(1)

Data fetched successfully!
Data fetched successfully!
Data fetched successfully!
Data fetched successfully!
Data fetched successfully!
Data fetched successfully!
Data fetched successfully!
Data fetched successfully!
Data fetched successfully!
Data fetched successfully!
Data fetched successfully!
Data fetched successfully!
Data fetched successfully!
Data fetched successfully!
Data fetched successfully!
Data fetched successfully!
Data fetched successfully!
Data fetched successfully!
Data fetched successfully!
Data fetched successfully!
Data fetched successfully!
Data fetched successfully!


In [4]:
# CLeaning linkedin prefixes
for idx,desc in enumerate(data['job_description']):
    if desc[:10] == 'Sign in to':
        data['job_description'][idx] = desc[1212:]
    elif desc[:12] == 'Remove photo':
        if desc[291:][:6] == 'Use AI':
            data['job_description'][idx] = desc[1642:]
        else:
            data['job_description'][idx] = desc[291:]
    else:
        data['job_description'][idx] = desc
       

In [5]:
df = pd.DataFrame(data)
# Check for duplicates
df = df.drop_duplicates(subset=['job_title', 'job_company', 'job_location', 'days_ago', 'company_description', 'job_description'], keep='first')
df.reset_index(drop=True, inplace=True)
df

,job_title,job_company,job_location,days_ago,company_description,job_description
0,Machine Learning Engineer,Gain.pro,"Amsterdam, North Holland, Netherlands",1,role at Gain.pro,Who are we? Gain.pro is on a mission to provid...
1,Data Scientist II - Smart Benefits Insights,Booking.com for Business,"Amsterdam, North Holland, Netherlands",2,role at Booking.com for Business,"CAROUSEL_PARAGRAPH Amsterdam, Netherlands Data..."
2,Data Scientist,Harnham,Amsterdam Area,6,Elisabeth Oudman Elisabeth Oudman,"Data Scientist Amsterdam, The Netherlands Up t..."
3,Data Scientist,Harnham,Amsterdam Area,6,Elisabeth Oudman Elisabeth Oudman,"Data Scientist Amsterdam, The Netherlands Up t..."
4,Data Scientist,KINESSO NL,"Amsterdam, North Holland, Netherlands",7,role at KINESSO NL,Direct message the job poster from KINESSO NL ...
5,Data Scientist AI/ML in Product Development,Stantec,"Arnhem, Gelderland, Netherlands",5,role at Stantec,Requisition Number: 23583BR Description Ben ji...
6,AI/Machine Learning Engineer,Inpact,"Hoofddorp, North Holland, Netherlands",1,role at Inpact,Direct message the job poster from Inpact Gijs...
7,Freelance data scientist,Harnham,"Amsterdam, North Holland, Netherlands",2,What You’ll Be Doing You’ll take on complex bu...,Business Collaboration Work closely with comme...
8,Data Scientist AI/ML in Product Development,Stantec,"Breda, North Brabant, Netherlands",5,role at Stantec,Requisition Number: 23583BR Description Ben ji...
9,Data Scientist bij OHRA,OHRA,"Arnhem, Gelderland, Netherlands",3,role at OHRA,Are you ready for the next step in your career...


In [1]:
#df.to_csv('linkedin_jobs.csv', index=False)
import pandas as pd
df = pd.read_csv('linkedin_jobs.csv')

In [2]:
import os
from huggingface_hub import hf_hub_download
HF_API_KEY = os.getenv('API_KEY')
model_id = "google/flan-t5-base"
file_names = ["pytorch_model.bin", "added_tokens.json", "config.json", "generation_config.json", "special_tokens_map.json", "spiece.model", "tokenizer_config.json"]

ModuleNotFoundError: No module named 'huggingface_hub'